In [29]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset
from torch.autograd import Variable
import torchvision
import pathlib
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt


In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/becode/Skybase/pytorch/pytenv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
device

device(type='cpu')

In [4]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [20]:
RANDOM_STATE = 42
# Data

DATA_ROOT = '/home/becode/Skybase/new_data'

TEST_SIZE = 0.2
TRAIN_BATCH = 64
VALID_BATCH = 100
IMG_SIZE = (150, 150)

In [31]:
dataset = ImageFolder(DATA_ROOT, transform=transformer, target_transform=None)


splitter = StratifiedShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=RANDOM_STATE)
splits = splitter.split(X=np.arange(len(dataset), dtype=np.int), y=dataset.targets)

train_idx, valid_idx = next(splits)

train_ds = Subset(dataset, train_idx)
valid_ds = Subset(dataset, valid_idx)

#train_loader = DataLoader(train_ds, batch_size=TRAIN_BATCH, shuffle=True,  num_workers=5)
#valid_loader = DataLoader(valid_ds, batch_size=VALID_BATCH, shuffle=False, num_workers=5)

train_loader=DataLoader(
    train_ds,
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    valid_ds,
    batch_size=32, shuffle=True
)


'''imgs = [
    train_ds[0][0].permute(1, 2, 0).reshape((3,150, 150)), 
    train_ds[-1][0].permute(1, 2, 0).reshape((3,150, 150))
]'''

'imgs = [\n    train_ds[0][0].permute(1, 2, 0).reshape((3,150, 150)), \n    train_ds[-1][0].permute(1, 2, 0).reshape((3,150, 150))\n]'

In [24]:
train_dl

def show_in_row(list_of_images: list, titles: list = None, disable_ticks: bool = False):
    count = len(list_of_images)
    for idx in range(count):
        subplot = plt.subplot(1, count, idx+1)
        if titles is not None:
            subplot.set_title(titles[idx])
      
        img = list_of_images[idx]
        cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
        subplot.imshow(img, cmap=cmap)
        if disable_ticks:
            plt.xticks([]), plt.yticks([])
    plt.show()

In [33]:
#categories
root=pathlib.Path(DATA_ROOT)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [34]:
classes

['Negative', 'Positive']

In [32]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [35]:
model=ConvNet(num_classes=2).to(device)

In [36]:
#Optmizer and loss function

optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)

loss_function=nn.CrossEntropyLoss()

In [37]:
num_epochs=10

In [41]:
train_count = len(train_ds)
train_count

32496

In [43]:
test_count = len(valid_ds)
test_count

8124

In [44]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(0.1564) Train Accuracy: 0.9875984736582964 Test Accuracy: 0.9110044313146234
Epoch: 1 Train Loss: tensor(0.0901) Train Accuracy: 0.9921528803545052 Test Accuracy: 0.9897833579517479
Epoch: 2 Train Loss: tensor(0.0956) Train Accuracy: 0.9923067454455933 Test Accuracy: 0.9873215164943377
Epoch: 3 Train Loss: tensor(0.0614) Train Accuracy: 0.9944300837026095 Test Accuracy: 0.9251600196947316
Epoch: 4 Train Loss: tensor(0.1030) Train Accuracy: 0.9908911866075825 Test Accuracy: 0.9892909896602659
Epoch: 5 Train Loss: tensor(0.0547) Train Accuracy: 0.9942146725750861 Test Accuracy: 0.9948301329394387
Epoch: 6 Train Loss: tensor(0.0261) Train Accuracy: 0.9969842442146726 Test Accuracy: 0.9929837518463811
Epoch: 7 Train Loss: tensor(0.0130) Train Accuracy: 0.9975689315608075 Test Accuracy: 0.9934761201378631
Epoch: 8 Train Loss: tensor(0.0197) Train Accuracy: 0.9973842934515017 Test Accuracy: 0.9945839487936977
Epoch: 9 Train Loss: tensor(0.0334) Train Accuracy: 0.9